In [49]:
:dep pcap-file = "2"

use std::fs::File; 

use pcap_file::pcapng::PcapNgReader; 
use pcap_file::pcapng::blocks::Block;
use pcap_file::pcapng::blocks::enhanced_packet::EnhancedPacketBlock;

In [ ]:
#[derive(Debug)]
struct UrbHeader {
    id: u64, 
    typ: UrbType, 
    transfer_type: TransferType, 
    endpoint: UsbEndpoint, 
    device_address: u8, 
    bus_number: u16, 
    setup_fragment: Option<SetupFragment>,
    data_fragment: Option<DataFragment>, 
    urb_status: UrbStatus, 
    transfer_flags: u32, 
}

#[derive(Debug)]
enum CommDirection {
    HostIn, 
    HostOut, 
}

#[derive(Debug)]
enum UrbType {
    ParsingError, 
    Submit, 
    Complete, 
    UrbError, 
}

#[derive(Debug)] 
enum TransferType {
    Isochronous, 
    Interrupt, 
    Control, 
    Bulk, 
}

#[derive(Debug)]
struct UsbEndpoint {
    raw: u8, 
    direction: CommDirection,
}

#[derive(Debug)]
struct SetupFragment {
    bmRequestType: u8, 
    bRequest: u8,
    wValue: u16, 
    wIndex: u16, 
    wLength: u16, 
}

#[derive(Debug)]
enum UrbStatus {
    Success,
    Err(i32),
}

fn parse_packet_urb(packet: EnhancedPacketBlock) {
    // 1. Use try_into().unwrap() to convert slices to fixed-size arrays
    let urb_id = u64::from_le_bytes(packet.data[0..8].try_into().unwrap());
    
    let urb_type = match packet.data[8] {
        0x53 => UrbType::Submit,
        0x43 => UrbType::Complete,
        _    => UrbType::UrbError,
    };

    let transfer_type = match packet.data[9] {
        0x00 => TransferType::Isochronous, 
        0x01 => TransferType::Interrupt, 
        0x02 => TransferType::Control, 
        0x03 => TransferType::Bulk,
        _    => TransferType::Bulk, // Added a fallback
    };

    let usb_endpoint = UsbEndpoint {
        raw: packet.data[10], 
        direction: if packet.data[10] & 0x80 != 0 {
            CommDirection::HostIn
        } else {
            CommDirection::HostOut
        }
    };

    let bus_number = u16::from_le_bytes(packet.data[12..14].try_into().unwrap());

    let setup_fragment = match packet.data[14] {
        0x00 => Some(SetupFragment {
            bmRequestType: packet.data[40], 
            bRequest: packet.data[41], 
            wValue: u16::from_le_bytes(packet.data[42..44].try_into().unwrap()),
            wIndex: u16::from_le_bytes(packet.data[44..46].try_into().unwrap()), 
            wLength: u16::from_le_bytes(packet.data[46..48].try_into().unwrap()), 
        }),
        _    => None,
    };
    let data_fragment = match packet.data[15] {
        0x00 => Some(parse_data_fragment(packet.clone())),
        _    => None,
    };

    let urb_status_value = i32::from_le_bytes(packet.data[28..32].try_into().unwrap()); 
    let urb_status = match urb_status_value {
        0 => UrbStatus::Success,
        e => UrbStatus::Err(e),
    };

    let transfer_flags = u32::from_le_bytes(packet.data[56..60].try_into().unwrap());
    
    let urb_header = UrbHeader {
        id: urb_id, 
        typ: urb_type, 
        transfer_type,
        endpoint: usb_endpoint, 
        device_address: packet.data[11], 
        bus_number,
        setup_fragment, 
        data_fragment,
        urb_status, 
        transfer_flags,
    };

    println!("{:#04x?}", urb_header);
}

#[derive(Debug)] 
enum DataFragment {
    Other(Vec<u8>), 
    SbProtocol(SbProtocol), 
}

#[derive(Debug)]
struct SbProtocol {
    family: InstructionFamily, 
}

#[derive(Debug)]
enum InstructionFamily {
    ParsingError(u8),
    GlobalFeature,        // 0x26 
    SbxFeature,           // 0x12 (?) 
}

#[derive(Debug)]
enum SbxFeature {
    ParsingError,
    
}

fn parse_data_fragment(packet: EnhancedPacketBlock) -> DataFragment {
    let data: &[u8] = &packet.data[64..];
    
    if data[0] != 0x5a { return DataFragment::Other(data.to_vec()) };

    let instruction_family: InstructionFamily = match data[1] {
        0x26 => InstructionFamily::GlobalFeature, 
        _    => InstructionFamily::ParsingError(data[1]),
    };

    println!("testy");
    
    todo!()
}

In [ ]:

fn parse_packet_data(packet: EnhancedPacketBlock) {
    println!("- Data Fragment:");
    
    let Some(data_fragment) = &packet.data.get(64..) else {
        println!("  - Error");
        return
    };
    if data_fragment.is_empty() {
        println!("  - None");
        return
    };

    let descriptor_bytes = &data_fragment[0..2]; 

    println!("Data Type:");
    println!("- Raw           : {:02x?}", descriptor_bytes);

    match descriptor_bytes {
        [0x12, 0x01] => {
            println!("  - Device Descriptor");
            return
        },
        [0x09, 0x02] => {
            println!("  - Configuration Descriptor");
            return
        },
        [0x5a, _] => {
            println!("- Soundblaster Protocol");
        },
        _ => {
            println!("  - Parsing Error");
            return
        },
    }

    let formatted = data_fragment.iter()
        .map(|b| format!("{:#04x}", b))
        .collect::<Vec<_>>()
        .join(", ");
    println!("  - Raw: [{}]", formatted);

    if data_fragment.len() < 3 { return; }

    let cmd = data_fragment[1];
    let len = data_fragment[2];

    println!("Soundblaster Instruction:");
    println!("- CMD {:#04x}  LEN {}", cmd, len);

    match (cmd, len) {
        // DeviceAck: 5a 02 0a [echoed_cmd] 00 [payload_echo...]
        (0x02, 0x0a) => {
            let echoed = data_fragment.get(3).copied().unwrap_or(0);
            println!("- DeviceAck (echoes cmd {:#04x})", echoed);
        },
        // DeviceIdentify
        (0x05, 0x00) => println!("- DeviceIdentify Request"),
        (0x05, 0x04) => {
            let flags = data_fragment.get(3).copied().unwrap_or(0);
            println!("- DeviceIdentify Response (flags: {:#04x})", flags);
        },
        // Ping: 5a 06 01 01
        (0x06, _) => println!("- Ping"),
        // GetFirmwareString
        (0x07, 0x01) => {
            let typ = data_fragment.get(3).copied().unwrap_or(0);
            println!("- GetFirmwareString Request (type: {:#04x})", typ);
        },
        (0x07, _) => {
            let end = (3 + len as usize).min(data_fragment.len());
            let s = String::from_utf8_lossy(&data_fragment[3..end]);
            println!("- GetFirmwareString Response: {:?}", s);
        },
        // GetSerial
        (0x10, 0x00) => println!("- GetSerial Request"),
        (0x10, _) => {
            let end = (3 + len as usize).min(data_fragment.len());
            println!("- GetSerial Response: {:02x?}", &data_fragment[3..end]);
        },
        // StatusRequest: 5a 11 03 01 [family] [feature_id]
        (0x11, 0x03) => {
            let family     = data_fragment.get(4).copied().unwrap_or(0);
            let feature_id = data_fragment.get(5).copied().unwrap_or(0);
            println!("- StatusRequest (read {:#04x}:{:#04x})", family, feature_id);
        },
        // Single-feature StatusResponse: 5a 11 08 01 00 [family] [feature_id] [f32_LE×4]
        (0x11, 0x08) => {
            println!("- StatusResponse (single feature)");
            if data_fragment.len() >= 11 {
                let protocol_bytes = &data_fragment[5..11];
                println!("  - Protocol Bytes: {:02x?}", protocol_bytes);
                let protocol = parse_protocol(protocol_bytes);
                println!("  - Protocol: {:?}", protocol);
            }
        },
        // Multi-feature StatusResponse: 5a 11 [len] [count] 00 ([family][id][f32_LE])×count
        // len formula: 0x02 + (count × 6)
        (0x11, _) => {
            let count = data_fragment.get(3).copied().unwrap_or(0);
            println!("- StatusResponse ({} features)", count);
        },
        // WriteSingleFeature: 5a 12 07 01 [family] [feature_id] 00 [f32_LE×4]
        (0x12, 0x07) => {
            println!("- WriteSingleFeature");
            if data_fragment.len() >= 11 {
                // Skip the 0x00 padding at byte 6; join [family, feature_id] with [f32×4]
                // so parse_protocol sees the same layout as StatusResponse
                let family_id = &data_fragment[4..6];
                let f32_bytes = &data_fragment[7..11];
                let protocol_bytes: Vec<u8> = [family_id, f32_bytes].concat();
                println!("  - Protocol Bytes: {:02x?}", protocol_bytes);
                let protocol = parse_protocol(&protocol_bytes);
                println!("  - Protocol: {:?}", protocol);
            }
        },
        // BulkRangeDump: 5a 15 [len] [unknown] [count] [entries...]
        // Each entry: family(1) id(1) max_f32(4) min_f32(4) step_f32(4) = 14 bytes
        (0x15, _) => {
            let count = data_fragment.get(4).copied().unwrap_or(0);
            println!("- BulkRangeDump ({} entries)", count);
        },
        // GetHardwareId
        (0x20, 0x00) => println!("- GetHardwareId Request"),
        (0x20, 0x04) => {
            let hw_id = data_fragment.get(3).copied().unwrap_or(0);
            println!("- GetHardwareId Response: {:#04x}", hw_id);
        },
        // GetDspVersion
        (0x30, 0x00) => println!("- GetDspVersion Request"),
        (0x30, _) => {
            let end = (3 + len as usize).min(data_fragment.len());
            println!("- GetDspVersion Response: {:02x?}", &data_fragment[3..end]);
        },
        // GlobalProfile: bitmask — bit0=SBX Master, bit1=Scout Mode, bit2=EQ Enable
        (0x26, _) => {
            println!("- GlobalProfile");
            match data_fragment.get(3).copied() {
                Some(0x08) => {
                    // Read response: 5a 26 0b 08 ff ff [bitmask] ...
                    if let Some(&bitmask) = data_fragment.get(6) {
                        println!("  - SBX Master : {}", if bitmask & 0x01 != 0 { "ON" } else { "OFF" });
                        println!("  - Scout Mode : {}", if bitmask & 0x02 != 0 { "ON" } else { "OFF" });
                        println!("  - EQ Enable  : {}", if bitmask & 0x04 != 0 { "ON" } else { "OFF" });
                    }
                },
                Some(0x07) => {
                    // Write: 5a 26 05 07 [feature_id] 00 [state] 00
                    let feature_id = data_fragment.get(4).copied().unwrap_or(0);
                    let state      = data_fragment.get(6).copied().unwrap_or(0);
                    let feature = match feature_id {
                        0x01 => "SBX Master",
                        0x02 => "Scout Mode",
                        _    => "Unknown",
                    };
                    println!("  - Write {} → {}", feature, if state != 0 { "ON" } else { "OFF" });
                },
                _ => {},
            }
        },
        // OutputSelect: direction 0x00=write, 0x01=read response, 0x02=enumerate
        (0x2c, _) => {
            let direction = data_fragment.get(3).copied().unwrap_or(0xff);
            println!("- OutputSelect");
            println!("  - Direction: {}", match direction {
                0x00 => "Write",
                0x01 => "Read Response",
                0x02 => "Enumerate",
                _    => "Unknown",
            });
            if direction == 0x00 || direction == 0x01 {
                let mode = data_fragment.get(4).copied().unwrap_or(0xff);
                println!("  - Output: {}", match mode {
                    0x02 => "Speakers",
                    0x04 => "Headphones",
                    _    => "Unknown",
                });
            }
        },
        // Capabilities / RGB lighting (sub-command at byte 3)
        (0x3a, _) => {
            let sub = data_fragment.get(3).copied().unwrap_or(0);
            println!("- Capabilities/RGB (sub: {:#04x})", sub);
            if sub == 0x06 {
                let state = data_fragment.get(4).copied().unwrap_or(0xff);
                println!("  - RGB: {}", match state { 0x01 => "ON", 0x00 => "OFF", _ => "?" });
            }
        },
        // GainConfig: request 5a 3c 02 01 00 → response 5a 3c 04 01 00 [gain] 00
        (0x3c, _) => {
            let direction = data_fragment.get(3).copied().unwrap_or(0xff);
            println!("- GainConfig (direction: {:#04x})", direction);
            if len == 0x04 {
                let gain = data_fragment.get(5).copied().unwrap_or(0);
                println!("  - Gain: {:#04x}", gain);
            }
        },
        // DacFilter: direction 0x00=write, 0x01=read response, 0x02=enumerate
        (0x6c, _) => {
            let direction = data_fragment.get(3).copied().unwrap_or(0xff);
            println!("- DacFilter");
            println!("  - Direction: {}", match direction {
                0x00 => "Write",
                0x01 => "Read Response",
                0x02 => "Enumerate",
                _    => "Unknown",
            });
            if direction == 0x00 || direction == 0x01 {
                let filter = data_fragment.get(4).copied().unwrap_or(0xff);
                println!("  - Filter: {}", match filter {
                    0x01 => "Fast Roll-off, Minimum Phase",
                    0x02 => "Slow Roll-off, Minimum Phase",
                    0x03 => "NOS (Non-Oversampling)",
                    0x04 => "Fast Roll-off, Linear Phase",
                    0x05 => "Slow Roll-off, Linear Phase",
                    _    => "Unknown",
                });
            }
        },
        // Notification: 5a 6e 02 01 00 (device→host) or 5a 6e 01 01 (host→device)
        (0x6e, _) => {
            println!("- Notification (sub: {:#04x})", data_fragment.get(3).copied().unwrap_or(0));
        },
        _ => {
            println!("- Unknown (cmd: {:#04x})", cmd);
        },
    }
}

#[derive(Debug)]
enum Protocol {
    ParsingError,
    DspFeature(DspFeature),
    UnknownFamily(u8),
}

fn parse_protocol(protocol_bytes: &[u8]) -> Protocol {
    if protocol_bytes.is_empty() { return Protocol::ParsingError; }
    match protocol_bytes[0] {
        0x96 => Protocol::DspFeature(parse_dsp_feature(protocol_bytes)),
        family => Protocol::UnknownFamily(family),
    }
}

#[derive(Debug)]
enum FeatureValue {
    ParsingError,
    Raw([u8; 4]),
    Parsed(f32),
}

/// Family 0x96 DSP effect features. Values are IEEE 754 f32 LE.
#[derive(Debug)]
enum DspFeature {
    // SBX Pro Studio sub-features
    SurroundToggle(FeatureValue),     // 0x00  bool
    SurroundLevel(FeatureValue),      // 0x01  0.0–1.0
    DialogPlusToggle(FeatureValue),   // 0x02  bool
    DialogPlusLevel(FeatureValue),    // 0x03  0.0–1.0
    SmartVolToggle(FeatureValue),     // 0x04  bool
    SmartVolLevel(FeatureValue),      // 0x05  0.0–1.0
    SmartVolMode(FeatureValue),       // 0x06  0.0=Normal, 1.0=Loud, 2.0=Night
    CrystalizerToggle(FeatureValue),  // 0x07  bool
    CrystalizerLevel(FeatureValue),   // 0x08  0.0–1.0
    // Equalizer
    EqToggle(FeatureValue),           // 0x09  bool
    EqPreAmp(FeatureValue),           // 0x0a  ±6.0 dB, step 0.5
    Eq31Hz(FeatureValue),             // 0x0b  ±12.0 dB, step 0.5
    Eq62Hz(FeatureValue),             // 0x0c  ±12.0 dB
    Eq125Hz(FeatureValue),            // 0x0d  ±12.0 dB
    Eq250Hz(FeatureValue),            // 0x0e  ±12.0 dB
    Eq500Hz(FeatureValue),            // 0x0f  ±12.0 dB
    Eq1kHz(FeatureValue),             // 0x10  ±12.0 dB
    Eq2kHz(FeatureValue),             // 0x11  ±12.0 dB
    Eq4kHz(FeatureValue),             // 0x12  ±12.0 dB
    Eq8kHz(FeatureValue),             // 0x13  ±12.0 dB
    Eq16kHz(FeatureValue),            // 0x14  ±12.0 dB
    // More SBX sub-features
    SurroundDistance(FeatureValue),   // 0x17  10.0–300.0, step 0.5
    BassToggle(FeatureValue),         // 0x18  bool
    BassLevel(FeatureValue),          // 0x19  0.0–1.0
    // Observed but purpose unknown
    Unknown(u8, FeatureValue),        // 0x15, 0x16, 0x1a–0x1d, 0x70–0x72
}

fn parse_dsp_feature(protocol_bytes: &[u8]) -> DspFeature {
    // protocol_bytes: [family(0x96), feature_id, f32_byte0, f32_byte1, f32_byte2, f32_byte3]
    // This layout is the same for both StatusResponse and WriteSingleFeature
    // (the caller strips the 0x00 padding from WriteSingleFeature before calling)
    if protocol_bytes.len() < 2 {
        return DspFeature::Unknown(0, FeatureValue::ParsingError);
    }
    let feature_id = protocol_bytes[1];
    let value = if protocol_bytes.len() >= 6 {
        FeatureValue::Parsed(f32::from_le_bytes([
            protocol_bytes[2],
            protocol_bytes[3],
            protocol_bytes[4],
            protocol_bytes[5],
        ]))
    } else {
        FeatureValue::ParsingError
    };

    match feature_id {
        0x00 => DspFeature::SurroundToggle(value),
        0x01 => DspFeature::SurroundLevel(value),
        0x02 => DspFeature::DialogPlusToggle(value),
        0x03 => DspFeature::DialogPlusLevel(value),
        0x04 => DspFeature::SmartVolToggle(value),
        0x05 => DspFeature::SmartVolLevel(value),
        0x06 => DspFeature::SmartVolMode(value),
        0x07 => DspFeature::CrystalizerToggle(value),
        0x08 => DspFeature::CrystalizerLevel(value),
        0x09 => DspFeature::EqToggle(value),
        0x0a => DspFeature::EqPreAmp(value),
        0x0b => DspFeature::Eq31Hz(value),
        0x0c => DspFeature::Eq62Hz(value),
        0x0d => DspFeature::Eq125Hz(value),
        0x0e => DspFeature::Eq250Hz(value),
        0x0f => DspFeature::Eq500Hz(value),
        0x10 => DspFeature::Eq1kHz(value),
        0x11 => DspFeature::Eq2kHz(value),
        0x12 => DspFeature::Eq4kHz(value),
        0x13 => DspFeature::Eq8kHz(value),
        0x14 => DspFeature::Eq16kHz(value),
        0x17 => DspFeature::SurroundDistance(value),
        0x18 => DspFeature::BassToggle(value),
        0x19 => DspFeature::BassLevel(value),
        id => DspFeature::Unknown(id, value),
    }
}



In [ ]:
// let file = File::open("set_output_headphones.pcapng").expect("File must exist");
let file = File::open("set_sbx_on.pcapng").expect("File must exist"); 
let mut reader = PcapNgReader::new(file).unwrap();


while let Some(block) = reader.next_block() {
    let block = block.unwrap();
    
    match block {
        Block::EnhancedPacket(packet) => { 
            // println!("Byte 65: {:#02x?}", packet.data.get(64));
            let Some(sbx_byte) = packet.data.get(64) else {
                // println!("Failed to get Byte 65");
                continue
            };
            if sbx_byte != &0x5a {
                // println!("Not an SBX Packet");
                continue
            }
            
            println!("===== packet ===== {:?} =====", packet.timestamp);
            
            println!("Original Length : {:?}", packet.original_len);
            println!("Actual Length   : {:?}", packet.data.len());
            // println!("Raw Bytes       : {:02x?}", packet.data);
            
            parse_packet_urb(packet.clone());
            parse_packet_data(packet.clone()); 
            
            
            println!("===== endpkt ===== ------------------ =====");
        },
        _ => {
            // println!("{:0x?}", block); 
        },
    }
    
    // println!("{:0x?}", block);
}

===== packet ===== 1768973.797846508s =====
Original Length : 128
Actual Length   : 128



thread '<unnamed>' (121396) panicked at src/lib.rs:219:5:
not yet implemented
stack backtrace:
   0: __rustc::rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic
   3: <unknown>
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
